In [1]:
const _ = require('lodash')
const traverse = require('traverse')
const sparqljs = require('sparqljs')

const eachFileInDir = require('./eachFileInDir')
const countInQueries = require('./countInQueries')
const namespaces = require('./namespaces')
const parser = new sparqljs.Parser(namespaces)
const Query = require('./Query')

const WIKI = 'data/dewiki'

## Valid Queries

In [3]:
var isValidQuery = (query) => {
    return true // will internally throw an exception when parsed
}

countInQueries(WIKI, isValidQuery)

data/dewiki 772 772


## OPTIONAL

In [4]:
var hasOptional = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'optional'
    }, false)
}
countInQueries(WIKI, hasOptional)

data/dewiki 772 76


## FILTER

In [5]:
var hasFilter = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'filter'
    }, false)
}
countInQueries(WIKI, hasFilter)

data/dewiki 772 79


## ORDER BY

In [6]:
var hasOrderBy = (query) => {
    return query.getParsed().order
}
countInQueries(WIKI, hasOrderBy)

data/dewiki 772 66


## DISTINCT

In [7]:
var hasDistinct = (query) => {
    return query.getParsed().distinct
}
countInQueries(WIKI, hasDistinct)

data/dewiki 772 138


## GROUP BY

In [8]:
var hasGroupBy = (query) => {
    return query.getParsed().group
}
countInQueries(WIKI, hasGroupBy)

data/dewiki 772 29


## VALUES

In [9]:
var hasValues = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'values'
    }, false)
}
countInQueries(WIKI, hasValues)

data/dewiki 772 11


## UNION

In [10]:
var hasUnion = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'union'
    }, false)
}
countInQueries(WIKI, hasUnion)

data/dewiki 772 292


## NOT EXIST

In [11]:
var hasNotExist = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.operator === 'notexists'
    }, false)
}
countInQueries(WIKI, hasNotExist)

data/dewiki 772 20


## BIND

In [12]:
var hasBind = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'bind'
    }, false)
}
countInQueries(WIKI, hasBind)

data/dewiki 772 20


## MINUS

In [13]:
var hasMinus = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'minus'
    }, false)
}
countInQueries(WIKI, hasMinus)

data/dewiki 772 28


## Subqueries

In [14]:
var hasSubQuery = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.level > 0 && this.notLeaf && node.type === 'query'
    }, false)
}
countInQueries(WIKI, hasSubQuery)

data/dewiki 772 111


## Multiple Subject Queries

In [15]:
var hasMultipleSubjects = (query) => {
    const objects = query.getObjects()
    
    return query.getSubjects().some((subject) => {
        return typeof subject === 'string' && subject.startsWith('?') && objects.includes(subject)
    })
}
countInQueries(WIKI, hasMultipleSubjects)

data/dewiki 772 143


## Property Path

In [16]:
var hasPropertyPath = (query) => {
    return query.getPredicates().some(predicate => predicate.type === 'path')
}
countInQueries(WIKI, hasPropertyPath)

data/dewiki 772 457


## Most Common Property Path Predicates

In [17]:
var predicates = []
eachFileInDir(WIKI, (query, resolve) => {
    var query = new Query(query, parser.parse(query))
    
    predicates = traverse(query.getPredicates().filter(predicate => predicate.type === 'path')).reduce((acc, node) => {
        if (typeof node === 'string' && node.includes('prop/direct/P')) {
            acc.push(node)
        }
        
        return acc
    }, predicates)
    
    resolve()
}).then(() => {
    console.log(_.countBy(predicates, _.identity))
})

Promise { <pending> }

{ 'http://www.wikidata.org/prop/direct/P279': 217,
  'http://www.wikidata.org/prop/direct/P31': 117,
  'http://www.wikidata.org/prop/direct/P131': 594,
  'http://www.wikidata.org/prop/direct/P150': 487,
  'http://www.wikidata.org/prop/direct/P1435': 3,
  'http://www.wikidata.org/prop/direct/P176': 2,
  'http://www.wikidata.org/prop/direct/P1001': 35,
  'http://www.wikidata.org/prop/direct/P527': 5,
  'http://www.wikidata.org/prop/direct/P361': 6,
  'http://www.wikidata.org/prop/direct/P5607': 2,
  'http://www.wikidata.org/prop/direct/P21': 1,
  'http://www.wikidata.org/prop/direct/P136': 2,
  'http://www.wikidata.org/prop/direct/P106': 3,
  'http://www.wikidata.org/prop/direct/P4552': 1,
  'http://www.wikidata.org/prop/direct/P706': 1,
  'http://www.wikidata.org/prop/direct/P159': 1,
  'http://www.wikidata.org/prop/direct/P276': 1,
  'http://www.wikidata.org/prop/direct/P27': 1,
  'http://www.wikidata.org/prop/direct/P41': 1,
  'http://www.wikidata.org/prop/direct/P411': 2 }


## Object Variable Referencing (TODO: how is this really called?)

In [18]:
// This is about queries with object variables that are references as objects in another triple.
// This functionality is needed for cyclic links to answer questions like "What movies have actors starring together with their children?

var hasObjectReference = (query) => {
    const objectCounts = _.countBy(query.getObjects(), _.identity)
    return _.size(_.pickBy(objectCounts, (count, obj) => count > 1 && _.startsWith(obj, '?'))) > 0
}
countInQueries(WIKI, hasObjectReference)

data/dewiki 772 296


## Relevant Object Variable References

In [19]:
var hasRelevantObjectReference = (query) => {
    const objectCounts = _.countBy(query.getObjects(), _.identity)
    const referencedObjects = _.pickBy(objectCounts, (count, obj) => count > 1 && _.startsWith(obj, '?'))
    const predicatesWithObj = query.traverse().reduce(function(acc, node) {

      if (typeof node == 'object' && node.object && referencedObjects[node.object]) acc.push(node.predicate)

      return acc
    }, [])
    const relevantPredicatesWithObj = predicatesWithObj.filter(predicate => {
        return traverse(predicate).reduce((acc, node) => {
            return acc || typeof node === 'string' && node.includes('http://www.wikidata.org/prop/direct/')
        }, false)
    })
    
    
    return _.size(relevantPredicatesWithObj) > 1
}
countInQueries(WIKI, hasRelevantObjectReference)

data/dewiki 772 275


## Cycles

In [20]:
var transitive = (graph) => { // silly
    do {
        var change = false
        
        for (var subject in graph) {
            var subjectReachability = graph[subject]
            for (var i = 0; i < graph[subject].length; i++) {
                if (graph[graph[subject][i]]) {
                    subjectReachability = subjectReachability.concat(graph[graph[subject][i]])
                }
            }
            if (_.uniq(subjectReachability).length > graph[subject].length) {
                change = true
                graph[subject] = _.uniq(subjectReachability)
            }
        }
    } while (change)
        
    return graph
}

var hasCycles = (graph) => {
    for (var subject in graph) {
        if (graph[subject].includes(subject)) return true
    }
    return false
}

var hasCycle = (query) => {
    var reachability = query.traverse().reduce((acc, node) => {
        if (node.subject && node.object.startsWith('?')) {
            if (!acc[node.subject]) acc[node.subject] = []
            acc[node.subject].push(node.object)
        }
        
        return acc
    }, {})

    return hasCycles(transitive(reachability))
}

countInQueries(WIKI, hasCycle)

data/dewiki 772 0


## wdt:P31/wdt:P279*

In [21]:
var hasInstanceOfSubclassOf = (query) => {
    return query.getPredicates().some(predicate => {
        return ( predicate.type === 'path'
            && predicate.pathType === '/'
            && predicate.items[0] === 'http://www.wikidata.org/prop/direct/P31'
            && predicate.items[1].pathType === '*'
            && predicate.items[1].items[0] === 'http://www.wikidata.org/prop/direct/P279' )
    })
}
countInQueries(WIKI, hasInstanceOfSubclassOf)

data/dewiki 772 113


## wdt:P279*

In [22]:
var hasSubclassOf = function hasSubclassOf(predicate) {
    if (predicate.type !== 'path') return false
    if (predicate.pathType === '*' && predicate.items.includes('http://www.wikidata.org/prop/direct/P279')) {
        return true
    }
    
    return predicate.items.reduce((any, predicate) => { return any || hasSubclassOf(predicate) }, false)
}
var hasInstanceOfSubclassOf = (query) => {
    return query.getPredicates().some(hasSubclassOf)
}
countInQueries(WIKI, hasInstanceOfSubclassOf)

data/dewiki 772 180


## Use of Qualifiers

In [23]:
var hasQualifiers = (query) => {
    return query.getRaw().includes('pq:P')
}

countInQueries(WIKI, hasQualifiers)

data/dewiki 772 37


## Use of References

In [24]:
var hasReferences = (query) => {
    return query.getRaw().includes('pr:P')
}

countInQueries(WIKI, hasReferences)

data/dewiki 772 1


## Sitelinks

In [25]:
var hasSitelinks = (query) => {
    return query.getRaw().includes('schema:about')
}

countInQueries(WIKI, hasSitelinks)

data/dewiki 772 44


In [26]:
var predicates = {}

eachFileInDir(WIKI, (query, resolve) => {
    var wikibasePredicates = query.match(/wikibase:\w+/g)
    if (wikibasePredicates) wikibasePredicates.forEach((predicate) => {
        if (predicates[predicate]) predicates[predicate]++
        else predicates[predicate] = 1
    })
    resolve()
}).then(() => {
    var tuples = []
    for (var predicate in predicates) tuples.push([predicate, predicates[predicate]])
    console.log(tuples.sort((a, b) => b[1] - a[1]).slice(0, 5))
})

Promise { <pending> }

[ [ 'wikibase:label', 64 ],
  [ 'wikibase:language', 64 ],
  [ 'wikibase:rank', 17 ],
  [ 'wikibase:DeprecatedRank', 17 ],
  [ 'wikibase:radius', 5 ] ]


## Features per Query

In [27]:
var allFeatures = [
    hasOptional, hasFilter, hasOrderBy,
    hasValues, hasUnion, hasMinus,
    hasSubQuery, hasMultipleSubjects, hasPropertyPath,
    hasSubclassOf, hasQualifiers, hasReferences,
    hasGroupBy
]
var featuresPerQuery = (new Array(12)).fill(0)

eachFileInDir(WIKI, (query, resolve) => {
    let features = 0
    allFeatures.forEach((hasFeature) => {
        if (hasFeature(new Query(query, parser.parse(query)))) features++
    })
    featuresPerQuery[features]++
    resolve()
}).then(() => {
    console.log(featuresPerQuery)
})

// EN: [ 432, 111, 248, 117, 71, 9, 3, 1, 0, 0, 0, 0 ]
// DE: [ 85,  451,  74,  40,  7, 3, 0, 9, 0, 0, 0, 0 ]

// Updated
// EN: [ 432, 101, 222, 103, 79, 40, 9, 4, 2, 0, 0, 0 ]
// DE: [ 84, 343, 170, 27, 26, 7, 3, 9, 0, 0, 0, 0 ]

Promise { <pending> }

[ 211, 54, 392, 45, 33, 15, 4, 18, 0, 0, 0, 0 ]


## Finding a subset of features that is sufficient for most queries

In [28]:
var allFeatures = [
    hasOptional, hasFilter, hasOrderBy,
    hasValues, hasUnion, hasMinus,
    hasSubQuery, 
    hasGroupBy, hasPropertyPath
]
var subset = [
    hasOptional, hasFilter,
    hasValues, hasOrderBy, hasPropertyPath
];
var queriesProduceable = 0

var isProduceableWithSubset = (query) => {
    let featuresUsed = 0
    let subsetFeaturesUsed = 0

    allFeatures.forEach((hasFeature) => {
        if (hasFeature(query)) featuresUsed++
    })
    subset.forEach((hasFeature) => {
        if (hasFeature(query)) subsetFeaturesUsed++
    })
    
    return featuresUsed === subsetFeaturesUsed
}

countInQueries(WIKI, isProduceableWithSubset)

data/dewiki 772 337


## Queries with Wikidata-URI subjects (not variables)

In [29]:
var hasURISubject = (query) => {
    return query.getSubjects().some(subject => subject.startsWith('http://www.wikidata.org/entity/Q'))
}

countInQueries(WIKI, hasURISubject)

data/dewiki 772 3
